AMUSE tutorial on modules and channels
====================

AMUSE is composed of domain-specific modules (i.e. the physics modules), which are often written in some native compiled language, and data-handling.
the latter tends to be confusing for novel users.
Data representation in AMUSE is often replicated.
So can there be a parameter *mass* to indicate a property of a star in a stellar-evolution code, in a gravitational dynamics code and in your user script. These three parameters *mass* can mean the same, or they can have a different meaning. This makes running amuse somewhat confusing at times.
Here we show how to formally separate these data streams.

But first we will enter the realm of modules.

In [ ]:
#Load in the amuse units module, the particle module and 
# some generator for producing some conditions.
from amuse.units import units, constants
from amuse.lab import Particles

We now like to generate the Solar system, as it was observed on April 5th 2063 using the [JPL emphemeris](https://ssd.jpl.nasa.gov/horizons.cgi#top).
Let's start with the inner most planet, Mercury, in the Sun's barycenter, which gives the following output:

*******************************************************************************
$$SOE
2474649.500000000 = A.D. 2063-Apr-05 00:00:00.0000 TDB 
 X =-2.689701945882702E-01 Y = 1.947622508089924E-01 Z = 4.148911797144442E-02
 VX=-2.230751806031045E-02 VY=-2.157655548570704E-02 VZ= 2.791759037949523E-04
 LT= 1.932846451835678E-03 RG= 3.346619889187973E-01 RR= 5.406475142917982E-03
 
We can use these to start an AMUSE particle set, as follows

In [ ]:
Sun = Particles(1)
Sun.mass = 1 | units.MSun
Sun.position = (0,0,0) | units.km
Sun.velocity = (0,0,0) | units.km/units.s
planets = Particles(8)
planets[0].mass = 3.302e+23 | units.kg # also according to JPL)
planets[0].position = (-2.689701945882702E-01,
                       1.947622508089924E-01,
                       4.148911797144442E-02) | units.au
planets[0].velocity = (-2.230751806031045E-02,
                       -2.157655548570704E-02,
                       2.791759037949523E-04) | units.au/units.yr

And continue doing this for the other 7 planets.
Then add the Sun and we have the Solar system's particle set.
It would be easier if there is a handy routine with the same effect, in particular because of frequently will be using the Solar system as some sort of template for a rather typical planetary system, or for specifically sutdying this planetary system. We therefore have a handy routine that gives allows us to initialize the Solar system.

In [ ]:
from amuse.ext.solarsystem import new_solar_system
sun_and_planets = new_solar_system()
#sun_and_planets = sun_and_planets[0].as_set()
print(sun_and_planets)

We adopted the module `ext.new_solar_system` to generate a ready-made solar system. You can also make a solar system that includes it's moons. For this you will have to use the routine 
'new_lunar_system', which is a module from the 'amuse.ic.solar_system_moons' package.

Now, plot the positions of the sun and planets in the orbital plane.

In 1984  
[Whitemire and Jackson](https://ui.adsabs.harvard.edu/abs/1984Natur.308..713W/abstract) and [Davies et al 1984](
https://ui.adsabs.harvard.edu/abs/1984Natur.308..715D)([preprint](https://escholarship.org/uc/item/9gm5c682)) argued that the Solar system was orbited by a hypothetical star called nemesis, named after the goddess
of Rhamnous who punished those who did not honor the Olympians.

This companion to the Sun would have an orbital separation of about 95,000 au, and due to its interaction with the Oort cloud would initiate periodic comet shouwers in the inner Solar system; punishing life on Earth for being atheistic?

Let's adopt the Solar system as we have just initialized and add Nemesis as an 0.2MSun main-sequence star in a 95,000 au orbit with an eccentricity of 0.7.

Let's construct tha binary system of the Sun (without planets) and Nemesis.

In [ ]:
msun = sun_and_planets.mass.sum()
mnemesis = 0.2 | units.MSun
a = 95000 | units.au
e = 0.7
from amuse.ext.orbital_elements import new_binary_from_orbital_elements
sun_and_nemesis = new_binary_from_orbital_elements(msun, mnemesis, 
                                          a, e, G=constants.G)
setattr(sun_and_nemesis, "name", ["Companion", "Nemesis"])
print(sun_and_nemesis)

Now, the sun in this binary-particle set and the sun in the Solar system particle-set do not have the same position.

In [ ]:
sun = sun_and_planets[sun_and_planets.name=="SUN"]
print(sun.position.in_(units.au))
print(sun_and_nemesis[sun_and_nemesis.mass>0.9|units.MSun].position.in_(units.au))

And, equally wise they will have different velocity.

however, we would like to adot these binary parameters for the Sun-Nemesis binary system. For this, we first move the entire Solar system to the Sun barycenter, move it to Nemesis' companion and add Nemesis.

In [ ]:
sun_and_planets.position -= sun.position
sun_and_planets.velocity -= sun.velocity
companion = sun_and_nemesis[sun_and_nemesis.name=="Companion"]
nemesis = sun_and_nemesis[sun_and_nemesis.name=="Nemesis"]
sun_and_planets.position += companion.position
sun_and_planets.velocity += companion.velocity
sun_and_planets.add_particle(sun_and_nemesis[1])
print(sun_and_planets)

You have created a hypothetical planetary system with the Sun, 8 planets and 1 dwarf planet (mimiking the Solar system), and a secondary star (Nemesis) in a wide and elliptic orbit.

Assignmnets and questions:
---------------

### Assignment 1:
To check the orbit of Nemesis around the Sun you can calculate the orbital elements from the Cartesian coordinates of the Sun and Nemesis. Perform this operation and check the orbital elements of the nemesis-Sun binary. 

### Question 1:
When performing this assignment, we found that the orbit between the Sun and Nemesis is slightly off from what we put in. The semi-major axis is about 95,606 au with an eccentricity of about 0.702. 

Explain why the orbit is not what you anticipated it to be, and what can you do to mitigate this?

### Assignment 2: 
Add 100 zero-mass Oort cloud object with a mass in circular orbits around the Sun with semi-major axes equally spaced between 10000au and 50000 au. Integrate those orbits for 10 full orbits of the Sun-Nemesis binary. Make a scatter plot of eccentricity as function of the semi-major axis for the initial and final Oort-cloud objects. Explain the figure.

### Question 3:
Do you think that Nemesis could be the cause of periodic cometary visitors?